In [1]:
#OLLAMAPool Server Prototype Code
import os
EndPoint_Queries=os.environ.get('EndPoint_Queries')
Endpoint_Results=os.environ.get('Endpoint_Results')
EndPoint_NodeStatus=os.environ.get('EndPoint_NodeStatus')

#Assert if the environment variables are set
if EndPoint_Queries is None:
    raise ValueError("EndPoint_Queries is not set")
if Endpoint_Results is None:
    raise ValueError("Endpoint_Results is not set")
if EndPoint_NodeStatus is None:
    raise ValueError("EndPoint_NodeStatus is not set")
print("Environment Variables are set OK")


Environment Variables are set OK


In [2]:
#Example Use case -> character description generator for fantasy RPG
#Takes a random dictionary of character traits and generates a written character description

races=["elf","human","dwarf","halfling","gnome"]
complections=["pale","dark","fair","ruddy","swarthy"]
heights=["tall","short","average"]
builds=["slender","stocky","muscular","portly"]
features=["sharp","soft","angular","round"]
eyes=["bright","dark","piercing","dull"]
hairs=["long","short","curly","straight","wavy"]
clothes=["fine","simple","elegant","gaudy","tattered"]
sophistication=["refined","crude","simple","complex"]
mannerisms=["graceful","clumsy","boisterous","quiet"]
virtues=["honest","loyal","brave","generous","kind"]


import random

#Come up with a random dictionary of character traits
def GenerateCharacterDescDictionary():
    traits={}
    race=random.choice(races)
    complection=random.choice(complections)
    height=random.choice(heights)
    build=random.choice(builds)
    feature=random.choice(features)
    eye=random.choice(eyes)
    hair=random.choice(hairs)
    clothe=random.choice(clothes)
    sophisticate=random.choice(sophistication)
    mannerism=random.choice(mannerisms)
    virtue=random.choice(virtues)
    traits={"race":race,"complection":complection,"height":height,"build":build,"feature":feature,"eye":eye,"hair":hair,"clothes":clothe,"sophistication":sophisticate,"mannerism":mannerism,"virtue":virtue}
    return traits

#Turn the dictionary into a line separated string
def GenerateCharacterDescList(traits):
    desc="They have the followng traits:\n"
    for key in traits:
        desc+=key+": "+traits[key]+"\n"
    return desc
    
            
    
#Test code
traits=GenerateCharacterDescDictionary()
print(GenerateCharacterDescList(traits))


They have the followng traits:
race: human
complection: swarthy
height: short
build: slender
feature: soft
eye: dull
hair: short
clothes: simple
sophistication: refined
mannerism: clumsy
virtue: loyal



In [4]:
#Produce sample character descriptions
totalRecords=10
requestTexts=[]
for i in range(totalRecords):
    traits=GenerateCharacterDescDictionary()
    requestTexts.append(GenerateCharacterDescList(traits))
requestTexts

['They have the followng traits:\nrace: halfling\ncomplection: ruddy\nheight: short\nbuild: muscular\nfeature: round\neye: bright\nhair: wavy\nclothes: tattered\nsophistication: simple\nmannerism: graceful\nvirtue: generous\n',
 'They have the followng traits:\nrace: gnome\ncomplection: dark\nheight: tall\nbuild: muscular\nfeature: round\neye: dull\nhair: short\nclothes: simple\nsophistication: crude\nmannerism: graceful\nvirtue: brave\n',
 'They have the followng traits:\nrace: elf\ncomplection: pale\nheight: tall\nbuild: stocky\nfeature: sharp\neye: dark\nhair: curly\nclothes: gaudy\nsophistication: simple\nmannerism: boisterous\nvirtue: honest\n',
 'They have the followng traits:\nrace: elf\ncomplection: dark\nheight: average\nbuild: portly\nfeature: soft\neye: dull\nhair: wavy\nclothes: elegant\nsophistication: crude\nmannerism: quiet\nvirtue: brave\n',
 'They have the followng traits:\nrace: gnome\ncomplection: fair\nheight: short\nbuild: slender\nfeature: sharp\neye: dark\nhair: 

In [5]:
#Busines Classes
import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage

class LLMRequest:
    def __init__(self,Model:str="",systemMessage:str="",query:str=""):
        self.UUID=str(uuid.uuid4())
        self.Model=Model
        self.systemMessage=systemMessage
        self.query=query

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMResult:
    def __init__(self,UUID:str="",result:str="",errorMsg:str="",timeDelta:str=""):
        self.UUID=UUID
        self.result=result
        self.errorMsg=errorMsg
        self.HasError=errorMsg!=""
        self.timeDelta=timeDelta

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMRequestResultPair():
    def __init__(self,request:LLMRequest,result:LLMResult=None):
        self.uuid=request.UUID
        self.request=request
        self.result=result


In [6]:
#Main OLLAMA Calling Function
import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage



#Post to a service bus queue    
def AzurePost_ServiceBus(connection_string: str, queue_name: str, json_payload):
    with ServiceBusClient.from_connection_string(connection_string) as client:
        sender = client.get_queue_sender(queue_name=queue_name)
        message = ServiceBusMessage(json.dumps(json_payload))
        with sender:
            sender.send_messages(message)
            print(f"Queued message: {json_payload['UUID']}")

#Take a list of queries and turn them into results
def BatchTransForm(Model:str,systemMessage:str,queries:List[str],DynamicUI:bool=True,WebUI:bool=False,ShowWebUI:bool=False)->List[str]:
    
    #Put every prompt into the LLM_Requests pool
    LLMrequests=[]
    for query in queries:
        request=LLMRequest(Model,systemMessage,query)
        LLMrequests.append(request)
        AzurePost_ServiceBus(EndPoint_Queries,"llm-queries",request.to_json())
    
    #Monitor the results queue
    return LLMrequests
    
    #Output the results
    
#Test code
systemPrompt="""You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. 
You have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).
Try to keep it within medieval fantasy RPG lore and appropriate to this context.
"""
queuedRequests=BatchTransForm("llama3.1",systemPrompt,requestTexts)



Queued message: 59a0fc49-ea31-4841-8913-3493804a61b5
Queued message: 48451cb8-de42-4e82-9487-4371830b0a56
Queued message: 6968ff34-f04f-494b-a4d7-f6c707d13e9c
Queued message: 407278ac-0635-4891-90df-62f26edeee2c
Queued message: 83f0c4b7-8c29-4a0a-bf6b-eb1db1a9169f
Queued message: 6b1f1903-1531-4ced-ae0e-2b939128c63e
Queued message: 1819fa8d-5cbd-4d86-9620-d781e5e257a1
Queued message: 07d60784-c07a-49b9-9282-c111dedadb2b
Queued message: a21f4485-d1e1-4300-ac3a-07c0176b1e34
Queued message: 1555f281-dcea-4de0-82fd-23585d1d2059


In [7]:
#Main Result Handling Loop
import time
from typing import List
from azure.servicebus import ServiceBusClient, ServiceBusMessage

#Turn queud requests into dictionary
queueDict={}
for request in queuedRequests:
    queueDict[request.UUID]=LLMRequestResultPair(request)
totalResults=0
invalidResults=0

def ProcessResult(result:LLMResult):
    global totalResults
    global invalidResults
    if result.UUID in queueDict:
        print(f"Result UUID: {result.UUID} found in queueDict")
        queueDict[result.UUID].result=result
        totalResults+=1
    else:
        print(f"Result UUID: {result.UUID} not found in queueDict")
        invalidResults+=1

# Replace with your connection string and queue name
queue_name = "llm-responses"
servicebus_client = ServiceBusClient.from_connection_string(conn_str=Endpoint_Results)

def receive_messages_from_queue():
    # Create a receiver for the queue
    with servicebus_client.get_queue_receiver(queue_name=queue_name) as receiver:
        print("Receiving messages from the queue...")
        
        # Receive messages in a batch, you can specify max_message_count as needed
        received_msgs = receiver.receive_messages(max_message_count=1, max_wait_time=5)

        for msg in received_msgs:
            # Print the message payload
            print(f"Received message: {str(msg)}")
            receiver.complete_message(msg)
            
            llmRequest=LLMResult()
            llmRequest.from_json(str(msg))
            #llmRequest.from_json(json.loads(str(msg)))
            ProcessResult(llmRequest)
            
            # Accept the message to remove it from the queue
            #receiver.complete_message(msg)

# Call the function to read from the queue
while totalResults<len(queueDict):
    receive_messages_from_queue()
    print(f"Total Results: {totalResults}, Invalid Results: {invalidResults}")



Receiving messages from the queue...
Receiving messages from the queue...
Receiving messages from the queue...
Receiving messages from the queue...
Received message: {"UUID": "d379df8a-d947-45be-8c84-b704bfbbf88a", "result": {"model": "llama3.1", "created_at": "2024-09-22T13:52:14.414767809Z", "message": {"role": "assistant", "content": "What a delightful combination of traits! Let me weave a tale around this intriguing character.\n\nName: Bramble Burrowes\n\nBackground:\n\nBramble was once the proud proprietor of a small, yet cozy tavern in the rolling hills of the countryside. \"The Red Stag Inn\" was renowned for its warm hearth, hearty ale, and the most delectable pastries this side of the Shire. Bramble, being the refined soul that he is, ensured that every patron felt like family, whether they were a traveling bard or a weary adventurer. His kind heart and boisterous mannerisms made him a beloved fixture in the community.\n\nHowever, fate had other plans. The wars between the nei

KeyboardInterrupt: 